In [4]:
from BaseScraper import BaseScraper

from bs4 import BeautifulSoup
import requests




In [5]:
class NeruipsScraper(BaseScraper):
    _PAPER_FILES_LINK = "https://papers.nips.cc/paper_files/paper/"

This guy made a notebook also

https://colab.research.google.com/drive/1u51Id90ML79UdZaKD23qglY0ZkEmmdwk?usp=sharing#scrollTo=metallic-aquatic

In [7]:

# Send an HTTP GET request to the URL
response = requests.get(NeruipsScraper._PAPER_FILES_LINK)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the links with href containing "/paper/"
    links = soup.find_all("a", href=lambda href: href and "/paper/" in href)

    # Extract and print the href attribute of each link
    for link in links:
        print(link["href"])
else:
    print("Failed to retrieve the HTML content.")



https://datasets-benchmarks-proceedings.neurips.cc/paper/2021
/paper_files/paper/2022
/paper_files/paper/2021
/paper_files/paper/2020
/paper_files/paper/2019
/paper_files/paper/2018
/paper_files/paper/2017
/paper_files/paper/2016
/paper_files/paper/2015
/paper_files/paper/2014
/paper_files/paper/2013
/paper_files/paper/2012
/paper_files/paper/2011
/paper_files/paper/2010
/paper_files/paper/2009
/paper_files/paper/2008
/paper_files/paper/2007
/paper_files/paper/2006
/paper_files/paper/2005
/paper_files/paper/2004
/paper_files/paper/2003
/paper_files/paper/2002
/paper_files/paper/2001
/paper_files/paper/2000
/paper_files/paper/1999
/paper_files/paper/1998
/paper_files/paper/1997
/paper_files/paper/1996
/paper_files/paper/1995
/paper_files/paper/1994
/paper_files/paper/1993
/paper_files/paper/1992
/paper_files/paper/1991
/paper_files/paper/1990
/paper_files/paper/1989
/paper_files/paper/1988
/paper_files/paper/1987


In [8]:
import requests
from lxml import etree
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

colors = sns.color_palette(n_colors=10)

!mkdir figures

import asyncio
import aiohttp

async def make_request(url, session):
    # async with aiohttp.ClientSession() as session:
    try:
        async with session.get(url) as response:
            return await response.text()
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))

ModuleNotFoundError: No module named 'lxml'

In [ ]:
async def f(year):
    req = requests.get(f'https://neurips.cc/Conferences/{year}/Schedule')
    tree = etree.HTML(req.text)

    posters = tree.findall(".//div[@class='maincard narrower poster']")
    print(f"There are {len(posters)} in NeurIPS {year}.")
    urls = []
    for poster in posters:
        id_ = poster.attrib['id'].split('_')[1]
        urls.append(f'https://neurips.cc/Conferences/{year}/ScheduleMultitrack?event={id_}')
    connector = aiohttp.TCPConnector(force_close=True)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [make_request(url, session) for url in urls]
        responses = await asyncio.gather(*tasks)
    all_information = []
    for url, response in tqdm(zip(urls[:10],responses)):
        if not response:
            continue
        print(url)
        print(response)
        poster_tree = etree.HTML(response)
        event = poster_tree.find(".//div[@class='eventName']")
        poster_title = event.text

        information = poster_tree.findall(".//div[@class='panel-body']")

        abstract = information[0].getchildren()[0].text
        authors_information = information[1].getchildren()

        authors = []

        for auth in authors_information:

            info = auth.getchildren()[0]

            name = info.text
            affiliation = ','.join([x.text for x in info.getchildren()]) if len(info.getchildren()) > 0 else None

            authors.append({
                'name': name,
                'affiliation': affiliation
            })

        all_information.append({
            'title': poster_title,
            'abstract': abstract,
            'authors': authors,
            "url": url
        })
    with open(f'neurips_statistics{year}.json', 'w') as f:
        json.dump(all_information, f, indent = 4)
# await f(2023)